# Run the final state algorithm on a ROOT HSDATA file

In [ ]:
gROOT->ProcessLine(".x LoadK2.C+");

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.

In [ ]:
 K2 fs("Beam","Beam");
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [ ]:
auto output = new HS::FiledTree("tt","testcuts.root");
fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

**Making your own cuts class** Here I use a new cuts class DeltaTimePCut. You can make your own cuts follwing the example notebook [DeltaTimePCut](MakeDeltaTimePCut.ipynb)

In [ ]:
ParticleCutsManager cutsman;
auto beamCut=make_shared<DeltaTimePCut>(1,3); //7ns delta time cut
auto otherCut=make_shared<DeltaTimePCut>(10,3); //10ns cut for all other particles
cutsman.AddParticleCut("Beam",beamCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined. 

In [ ]:
fs.RegisterPostTopoAction(&cutsman);

**Particle Monitoring** here we are going to make seperate trees for each topology and only keep the detected particle information in each one.

In [ ]:
TreePrepManager treeman("/scratch/dglazier/g8/particles/t3");
treeman.SetParticleVars("Beam",{"P","Time"}); //For beam only interesetef in energy and time
treeman.ConfigureTreeParticles(&fs); //propogate through topologies
fs.RegisterPostWorkAction(&treeman); //register post-work i.e. after kinematics is called and all calculations

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [ ]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/w/work1/jlab/hallb/HS_g8/1_9_list/flist_048088.dat.root");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;

fs.ProcessData(); //No number give, analyse all events in chain
cout<<"Done "<<dm->GetEntry()<< " events "<<endl;

And remember to save the tree!

In [ ]:
//delete output;

In [ ]:
%jsroot on

In [ ]:
//TCanvas c3;
//c3.Divide(2,2);
c3.cd(1);
cout<<" rh "<<treeman.GetPrep(1)<<endl;
treeman.GetPrep(0)->Tree()->Draw("BeamP>>h1(100,0,3)","");
c3.cd(2);
treeman.GetPrep(0)->Tree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,4)","","col1");
c3.cd(3);
treeman.GetPrep(0)->Tree()->Draw("KpP:KpTime>>h3(100,-10,10,100,0,4)","","col1");
c3.cd(4);
treeman.GetPrep(0)->Tree()->Draw("KpP:KpTime>>h4(100,-10,10,100,0,4)","","col1");


In [ ]:
c3.Draw();

**When ready delete the treeman so trees as saved to the output directory**

In [ ]:
//delete &treeman;